In [2]:
%%capture
!pip install datasets rouge-score nltk transformers sentencepiece

In [57]:
#GPU check
!nvidia-smi

Thu Mar 31 19:50:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    37W / 250W |   4139MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
torch.cuda.empty_cache()

In [3]:
# Make sure your version of Transformers is at 4.11.0
# to run the following code correctly:
import datasets
import transformers
import torch

In [4]:
torch_device ='cuda'
#torch_device = 'cpu' (run this line if no gpu attached)

In [5]:
#load original model
from transformers import BartTokenizer, BartForConditionalGeneration

In [6]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to(torch_device)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

In [7]:
#prefix = "summarize: "
prefix = ""

##XSUM

In [8]:
#load test dataset
test_dataset = datasets.load_dataset('xsum', split='test')

Using custom data configuration default


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


In [9]:
small_test = test_dataset.select(list(range(2000)))
small_test

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 2000
})

In [10]:
# generate summary
batch_size = 16
max_input_length=512
max_target_length=64

def generate_summary(batch):
    sentences = [prefix + text for text in batch['document']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [11]:
results = small_test.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["document"])

  0%|          | 0/125 [00:00<?, ?ba/s]

In [12]:
results

Dataset({
    features: ['summary', 'id', 'pred'],
    num_rows: 2000
})

In [13]:
metric = datasets.load_metric("rouge")
rouge_score = metric.compute(predictions=results["pred"], references=results["summary"], use_stemmer=True)
# Extract a few results
rouge_mean = {key: value.mid.fmeasure * 100 for key, value in rouge_score.items()}

In [14]:
rouge_mean

{'rouge1': 14.595779093221637,
 'rouge2': 1.43984623968673,
 'rougeL': 11.4292131664307,
 'rougeLsum': 11.424532733096951}

In [22]:
!ls 

bart-base-finetuned-xsum.zip  drive  sample_data


In [19]:
#load fine-tuned model
!cp 'drive/MyDrive/weights/bart-base-finetuned-xsum.zip' '/content'

In [23]:
!unzip bart-base-finetuned-xsum.zip

Archive:  bart-base-finetuned-xsum.zip
   creating: content/bart-base-finetuned-xsum/checkpoint-25506/
  inflating: content/bart-base-finetuned-xsum/checkpoint-25506/scaler.pt  
  inflating: content/bart-base-finetuned-xsum/checkpoint-25506/merges.txt  
  inflating: content/bart-base-finetuned-xsum/checkpoint-25506/scheduler.pt  
  inflating: content/bart-base-finetuned-xsum/checkpoint-25506/tokenizer_config.json  
  inflating: content/bart-base-finetuned-xsum/checkpoint-25506/optimizer.pt  
  inflating: content/bart-base-finetuned-xsum/checkpoint-25506/config.json  
  inflating: content/bart-base-finetuned-xsum/checkpoint-25506/pytorch_model.bin  
  inflating: content/bart-base-finetuned-xsum/checkpoint-25506/vocab.json  
  inflating: content/bart-base-finetuned-xsum/checkpoint-25506/tokenizer.json  
  inflating: content/bart-base-finetuned-xsum/checkpoint-25506/special_tokens_map.json  
  inflating: content/bart-base-finetuned-xsum/checkpoint-25506/rng_state.pth  
  inflating: conten

In [24]:
finetuned_model = BartForConditionalGeneration.from_pretrained('content/bart-base-finetuned-xsum/checkpoint-25506/').to(torch_device)

In [25]:
# generate summary

def finetuned_summary(batch):
    sentences = [prefix + text for text in batch['document']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = finetuned_model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [26]:
finetuned_results = small_test.map(finetuned_summary, batched=True, batch_size=batch_size, remove_columns=["document"])

  0%|          | 0/125 [00:00<?, ?ba/s]

In [27]:
finetuned_score = metric.compute(predictions=finetuned_results["pred"], references=finetuned_results["summary"], use_stemmer=True)
# Extract a few results
finetuned_rouge_mean = {key: value.mid.fmeasure * 100 for key, value in finetuned_score.items()}

In [28]:
rouge_mean

{'rouge1': 14.595779093221637,
 'rouge2': 1.43984623968673,
 'rougeL': 11.4292131664307,
 'rougeLsum': 11.424532733096951}

In [30]:
finetuned_rouge_mean

{'rouge1': 36.878555111814,
 'rouge2': 15.625294876010193,
 'rougeL': 30.222456849914504,
 'rougeLsum': 30.23610072103022}

In [31]:
for i in range(20, 30):
    print("Original Text: %s" % small_test[i]['document'])
    print("\nActual Summary: %s" % results[i]['summary'])
    print("\nOriginal Prediction : %s" % results[i]['pred'])
    print("\nfinetuned Prediction : %s" % finetuned_results[i]["pred"])
    print("=====================================================================\n")

Original Text: Pakistan's telecoms regulator said the ban was no longer necessary because Google, which owns YouTube, had now launched a Pakistan-specific version.
YouTube has denied claims that the authorities can filter content.
Many young Pakistanis have welcomed the lifting of the ban but some activists want details of the deal with Google.
They say there should be greater transparency of the terms agreed between Google and the government.
A Pakistan Telecommunication Authority (PTA) official confirmed to the BBC that all internet service providers had been directed to open access to YouTube.
The Pakistan Telecommunication Company Ltd posted on its Facebook page on Monday: "Welcome Back YouTube".
Pakistan's ministry of information technology said: "Google has provided an online web process through which requests for blocking access of offending material can be made by the PTA to Google directly.
"Google/YouTube will accordingly restrict access to the said offending material for use

##CNN

In [32]:
#load test dataset
test_cnn = datasets.load_dataset('cnn_dailymail', '3.0.0', split='test')

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234. Subsequent calls will reuse this data.


In [33]:
small_test = test_cnn.select(list(range(2000)))
small_test

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 2000
})

In [34]:
# generate summary
batch_size = 16
max_input_length=512
max_target_length=128

def generate_summary(batch):
    sentences = [prefix + text for text in batch['article']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [35]:
results = small_test.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["article"])

  0%|          | 0/125 [00:00<?, ?ba/s]

In [36]:
results

Dataset({
    features: ['highlights', 'id', 'pred'],
    num_rows: 2000
})

In [37]:
rouge_score = metric.compute(predictions=results["pred"], references=results["highlights"], use_stemmer=True)
# Extract a few results
rouge_mean = {key: value.mid.fmeasure * 100 for key, value in rouge_score.items()}

In [38]:
rouge_mean

{'rouge1': 18.808710620084735,
 'rouge2': 6.003490723933439,
 'rougeL': 14.575350956096406,
 'rougeLsum': 16.851026586132722}

In [39]:
#load fine-tuned model
!cp 'drive/MyDrive/weights/bart-base-finetuned-cnn.zip' '/content'

In [40]:
!unzip bart-base-finetuned-cnn.zip

Archive:  bart-base-finetuned-cnn.zip
   creating: content/bart-base-finetuned-cnn/checkpoint-35890/
  inflating: content/bart-base-finetuned-cnn/checkpoint-35890/scaler.pt  
  inflating: content/bart-base-finetuned-cnn/checkpoint-35890/merges.txt  
  inflating: content/bart-base-finetuned-cnn/checkpoint-35890/scheduler.pt  
  inflating: content/bart-base-finetuned-cnn/checkpoint-35890/tokenizer_config.json  
  inflating: content/bart-base-finetuned-cnn/checkpoint-35890/optimizer.pt  
  inflating: content/bart-base-finetuned-cnn/checkpoint-35890/config.json  
  inflating: content/bart-base-finetuned-cnn/checkpoint-35890/pytorch_model.bin  
  inflating: content/bart-base-finetuned-cnn/checkpoint-35890/vocab.json  
  inflating: content/bart-base-finetuned-cnn/checkpoint-35890/tokenizer.json  
  inflating: content/bart-base-finetuned-cnn/checkpoint-35890/special_tokens_map.json  
  inflating: content/bart-base-finetuned-cnn/checkpoint-35890/rng_state.pth  
  inflating: content/bart-base-f

In [41]:
finetuned_model = BartForConditionalGeneration.from_pretrained('content/bart-base-finetuned-cnn/checkpoint-35890/').to(torch_device)

In [42]:
# generate summary

def finetuned_summary(batch):
    sentences = [prefix + text for text in batch['article']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = finetuned_model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [43]:
finetuned_results = small_test.map(finetuned_summary, batched=True, batch_size=batch_size, remove_columns=["article"])

  0%|          | 0/125 [00:00<?, ?ba/s]

In [44]:
finetuned_score = metric.compute(predictions=finetuned_results["pred"], references=finetuned_results["highlights"], use_stemmer=True)
# Extract a few results
finetuned_rouge_mean = {key: value.mid.fmeasure * 100 for key, value in finetuned_score.items()}

In [45]:
rouge_mean

{'rouge1': 18.808710620084735,
 'rouge2': 6.003490723933439,
 'rougeL': 14.575350956096406,
 'rougeLsum': 16.851026586132722}

In [46]:
finetuned_rouge_mean

{'rouge1': 25.19392797188557,
 'rouge2': 11.799069917155155,
 'rougeL': 20.962939717660735,
 'rougeLsum': 23.467100125313614}

In [47]:
for i in range(0, 15):
    print("Original Text: %s" % small_test[i]["article"])
    print("\nActual Summary: %s" % results[i]["highlights"])
    print("\nOriginal Prediction : %s" % results[i]['pred'])
    print("\nfinetuned Prediction : %s" % finetuned_results[i]["pred"])
    print("=====================================================================\n")

Original Text: (CNN)James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV's "The Dukes of Hazzard," died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he'd been a busy actor for decades in theater and in Hollywood, Best didn't become famous until 1979, when "The Dukes of Hazzard's" cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons, Best's Rosco P. Coltrane chased the moonshine-running Duke boys back and forth across the back roads of fictitious Hazzard County, Georgia, although his "hot pursuit" usually ended with him crashing his patrol car. Although Rosco was slow-witted and corrupt, Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive "kew-kew-kew" chuckle and for goofy catchphras

##Wikihow

In [48]:
#load test dataset
test_wiki = datasets.load_dataset("wikihow", "all", "/content/drive/MyDrive/dataset", split='test')

Using custom data configuration all-6f5101161f12f62f


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/157252 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5599 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5577 [00:00<?, ? examples/s]

Dataset wikihow downloaded and prepared to /root/.cache/huggingface/datasets/wikihow/all-6f5101161f12f62f/1.2.0/5343fc81d685acaa086c9cc19eb8706206cd1f8b315792b04c1d7b92091c305e. Subsequent calls will reuse this data.


In [49]:
small_test = test_wiki.select(list(range(2000)))
small_test

Dataset({
    features: ['text', 'headline', 'title'],
    num_rows: 2000
})

In [50]:
# generate summary
batch_size = 16
max_input_length=512
max_target_length=128

def generate_summary(batch):
    sentences = [prefix + text for text in batch['text']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [51]:
results = small_test.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["text"])

  0%|          | 0/125 [00:00<?, ?ba/s]

In [52]:
results

Dataset({
    features: ['headline', 'title', 'pred'],
    num_rows: 2000
})

In [53]:
rouge_score = metric.compute(predictions=results["pred"], references=results["headline"], use_stemmer=True)
# Extract a few results
rouge_mean = {key: value.mid.fmeasure * 100 for key, value in rouge_score.items()}

In [54]:
rouge_mean

{'rouge1': 15.760927403084057,
 'rouge2': 3.201758491359699,
 'rougeL': 12.046657148755866,
 'rougeLsum': 14.392227996841694}

In [ ]:
!ls 

content				   prophetnet-large-uncased-finetuned-wiki.zip
drive				   prophetnet-large-uncased-finetuned-xsum.zip
prophetnet-finetuned-cnn-half.zip  sample_data


In [55]:
#load fine-tuned model
!cp '/content/drive/My Drive/weights/bart-base-finetuned-wiki.zip' '/content'

In [56]:
!unzip bart-base-finetuned-wiki.zip

Archive:  bart-base-finetuned-wiki.zip
   creating: content/bart-base-finetuned-wiki/checkpoint-19657/
  inflating: content/bart-base-finetuned-wiki/checkpoint-19657/scaler.pt  
  inflating: content/bart-base-finetuned-wiki/checkpoint-19657/merges.txt  
  inflating: content/bart-base-finetuned-wiki/checkpoint-19657/scheduler.pt  
  inflating: content/bart-base-finetuned-wiki/checkpoint-19657/tokenizer_config.json  
  inflating: content/bart-base-finetuned-wiki/checkpoint-19657/optimizer.pt  
  inflating: content/bart-base-finetuned-wiki/checkpoint-19657/config.json  
  inflating: content/bart-base-finetuned-wiki/checkpoint-19657/pytorch_model.bin  
  inflating: content/bart-base-finetuned-wiki/checkpoint-19657/vocab.json  
  inflating: content/bart-base-finetuned-wiki/checkpoint-19657/tokenizer.json  
  inflating: content/bart-base-finetuned-wiki/checkpoint-19657/special_tokens_map.json  
  inflating: content/bart-base-finetuned-wiki/checkpoint-19657/rng_state.pth  
  inflating: conten

In [58]:
finetuned_model = BartForConditionalGeneration.from_pretrained('content/bart-base-finetuned-wiki/checkpoint-19657/').to(torch_device)

In [59]:
# generate summary

def finetuned_summary(batch):
    sentences = [prefix + text for text in batch['text']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = finetuned_model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [60]:
finetuned_results = small_test.map(finetuned_summary, batched=True, batch_size=batch_size, remove_columns=["text"])

  0%|          | 0/125 [00:00<?, ?ba/s]

In [61]:
finetuned_score = metric.compute(predictions=finetuned_results["pred"], references=finetuned_results["headline"], use_stemmer=True)
# Extract a few results
finetuned_rouge_mean = {key: value.mid.fmeasure * 100 for key, value in finetuned_score.items()}

In [62]:
rouge_mean

{'rouge1': 15.760927403084057,
 'rouge2': 3.201758491359699,
 'rougeL': 12.046657148755866,
 'rougeLsum': 14.392227996841694}

In [63]:
finetuned_rouge_mean

{'rouge1': 25.394797540689474,
 'rouge2': 10.31141781555491,
 'rougeL': 21.567606918626105,
 'rougeLsum': 24.794669692309384}

In [64]:
for i in range(20, 30):
    print("Original Text: %s" % small_test[i]['text'])
    print("\nActual Summary: %s" % results[i]['headline'])
    print("\nOriginal Prediction : %s" % results[i]['pred'])
    print("\nfinetuned Prediction : %s" % finetuned_results[i]["pred"])
    print("=====================================================================\n")

Original Text: In general, dwarf hamsters have thickset bodies, large cheek pouches, and short tails. Make sure it does not have any nasal or eye discharge, nor any other signs of illness. Russian dwarf hamsters are particularly prone to diabetes. A hamster suffering from this condition will drink a lot of water and urinate more frequently than other dwarf hamsters.Be sure to consult your veterinarian if you suspect that your hamster has diabetes.
 A recently weaned or an extremely stressed out dwarf hamster may come down with a disease called “wet tail.” Your hamster experiences diarrhea — the excessive moisture from this causes its tail to become literally wet. Consult a veterinarian for a proper diagnosis and treatment.


Tyzzer’s disease causes diarrhea in young or stressed hamsters. This is a disease that needs veterinarian treatment. Certain antibiotics can cause and exacerbate this condition, so don’t treat your hamster on your own., Like dogs and cats, dwarf hamsters can suffer